In [1]:
import pandas as pd
import sqlalchemy as sa
from sqlalchemy import text
import urllib
import json
from py2neo import Graph

with open("config.json") as json_data_file:
    data = json.load(json_data_file)

# cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+data["server"]+';DATABASE='+data["database"]+';ENCRYPT=no;UID='+data["username"]+';PWD='+data["password"])
# cursor = cnxn.cursor()

params = urllib.parse.quote_plus("DRIVER={SQL Server Native Client 11.0};"
                                 "SERVER="+data["server"]+";"
                                 "DATABASE="+data["database"]+";"
                                 "Trusted_Connection=yes")

engine = sa.create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))

# neo4j connection
port = "7687"
user = "neo4j"
pswd = "salmon"
# pswd = "trialcomposer2023"

# Make sure the database is started first, otherwise attempt to connect will fail
try:
    graph = Graph('bolt://localhost:'+port, auth=(user, pswd))
    print('SUCCESS: Connected to the Neo4j Database.')
except Exception as e:
    print('ERROR: Could not connect to the Neo4j Database. See console for details.')
    raise SystemExit(e)

SUCCESS: Connected to the Neo4j Database.


In [8]:
# read recipes table
recipes = pd.read_sql_query("SELECT * FROM recipe;", engine)
ingredient = pd.read_sql_query("SELECT * FROM ingredient;", engine)
step = pd.read_sql_query("SELECT * FROM cooking_step;", engine)

# create recipes nodes
for row in recipes.itertuples():
    graph.run('''
      CREATE (:Recipe {id: $id, name: $name})
    ''', parameters = {'id': row.id, 'name': row.name})

# create ingredient nodes
for row in step.itertuples():
    graph.run('''
      CREATE (:Step {id: $id, name: $name})
    ''', parameters = {'id': row.id, 'name': row.name})

# create ingredient nodes
for row in ingredient.itertuples():
    graph.run('''
      CREATE (:Ingredient {id: $id, name: $name})
    ''', parameters = {'id': row.id, 'name': row.name})

# create edges from recipe to ingredient
for row in recipes.itertuples():
    row_str = [row.ingredients]
    ingredient_dict_list = [json.loads(idx.replace("'", '"')) for idx in row_str]
    ingrdient_list = [d['id'] for d in ingredient_dict_list[0]]
    for ing in ingrdient_list:
        graph.run(f'''
            MATCH (r:Recipe), (i:Ingredient) 
            WHERE r.id = {row.id} AND i.id = {ing}
            CREATE (r)-[: HAS_INGREDIENTS]->(i) 
            ''')

# create edges from recipe to step
for row in recipes.itertuples():
    row_str = [row.steps]
    step_dict_list = [json.loads(idx.replace("'", '"')) for idx in row_str]
    step_list = [d['id'] for d in step_dict_list[0]]
    for stp in step_list:
        graph.run(f'''
            MATCH (r:Recipe), (s:Step) 
            WHERE r.id = {row.id} AND s.id = {stp}
            CREATE (r)-[: HAS_STEPS]->(s) 
            ''')